# Easily export jupyter cells to python module
https://github.com/fastai/course-v3/blob/master/nbs/dl2/notebook2script.py

In [1]:
! python /tf/main/src/scripts/notebook2script.py proc.ipynb

Converted proc.ipynb to exp/nb_proc.py


In [1]:
#export
from pathlib import Path
from fastai.text import *

import pandas as pd
import sentencepiece as sp

import csv

In [2]:
! pip install sklearn

You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import sys
sys.path.append("../")
from prep.exp.nb_prep import *

In [ ]:
#export
def df_to_txt_file(df, output):
    with open(output/'text.txt', 'w') as f:
        f.write('\n'.join(list(merged_trn_df["query"]) + list(merged_trn_df["res"])))
    return output/'text.txt'

In [ ]:
#export
def gen_sp_model(df, output):
    fname = df_to_txt_file(df, output)
    sp.SentencePieceTrainer.train(f'--input={fname} --model_prefix={output}/model')

In [ ]:
#export
class SentencePiece:
    "SentencePiece wrapper."
    def __init__(self, mdl_path):
        self.mdl_path = mdl_path
        self.s = sp.SentencePieceProcessor()
        self.s.Load(str(mdl_path))
        self.vocab = [self.s.IdToPiece(id) for id in range(self.s.GetPieceSize())]
        
    def tokenize(self, inpt):
        return self.s.EncodeAsPieces(inpt)

In [ ]:
s = sp.SentencePieceProcessor()
s.Load(str(data_path/"merged/model.model"))

In [ ]:
s.SampleEncodeAsPieces??

In [ ]:
len(s)

In [ ]:
for n in range(20):
    print(len(s.EncodeAsIds('public static void adfaj')))

In [ ]:
data_path = Path("/tf/data/datasets")

In [ ]:
merged_trn_df = pd.read_csv(data_path/"merged/trn.csv")
merged_trn_df.head()

In [ ]:
df_to_txt_file(merged_trn_df, data_path/"merged")

In [ ]:
gen_sp_model(merged_trn_df, data_path/"merged")

In [ ]:
merged_trn_df.head(1).to_string(header = False, index = False)

In [ ]:
print("yo")

In [ ]:
merged_trn_df?

In [ ]:
with open(data_path/'merged/text.txt', 'w') as f:
    f.write('\n'.join(list(merged_trn_df.head(2)["query"]) + list(merged_trn_df.head(2)["res"])))

In [ ]:
len(list(merged_trn_df.head(2)["query"]) + list(merged_trn_df.head(2)["res"]))

In [24]:
#export
def merge_cols(df):
    merged = [''.join(x) for x in zip(df["query"], df["res"])]
    new_df = pd.DataFrame({"merged": merged})
    
    return new_df

In [47]:
#export
def conv_to_ds(df, data_path, bs = 64):
#     print(df.head(5))
    return (TextList
            .from_df(df, data_path, 
                     processor = SPProcessor(
                         sp_model = data_path/"merged/model.model",
                         sp_vocab = data_path/"merged/model.vocab"
                     ))
            .split_none()
            .label_for_lm()
            .databunch(bs = bs)
           )

In [16]:
data_path = Path("/tf/data/datasets")
df_trn, df_val, df_tst = read_data(data_path/"merged")

In [48]:
db_trn = conv_to_ds(df_trn[:10], data_path)

In [50]:
ll = LabelLists(db_trn.path, db_trn.label_list.train, db_trn.label_list.valid)
ll

LabelLists;

Train: LabelList (10 items)
x: LMTextList
▁x x bo s ▁public ▁void ▁return result s ( java . util . array list < com . example . p it ur . new track om m ender . track info > ▁query result s ) ▁{ ▁if ▁( query result s ▁!= ▁null ) ▁{ ▁this . query result s ▁= ▁query result s ; ▁adapter ▁= ▁new ▁com . example . p it ur . new track om m ender . track info adapter ( this , ▁x x ma j ▁r . layout . list _ item , ▁query result s ); ▁set list adapter ( adapter ); ▁set item click listener s ( this . get list view ()); ▁} else ▁{ ▁android . widget . to ast . make text ( this , ▁" your ▁query ▁g ave ▁no ▁results . ", ▁to ast . length _ long ) . show (); ▁} ▁}<$ b ug $>,▁x x bo s ▁public ▁static ▁synchronized ▁json object ▁handling ex change ( java . util . array list < java . lang . string > ▁server list , ▁java . util . array list < java . lang . string > ▁server list _ ex change , ▁java . util . array list < java . lang . string > ▁host name list _ ex change , ▁java . util . array l

In [41]:
#export
def gen_lm_data(df_trn, df_val, task_name, data_path, bs = 64, sample = 1):
    if task_name != "merged":
        df_trn = tag_task(df_trn, task_name)
        df_val = tag_task(df_val, task_name)
    
    df_trn = df_trn.sample(frac = sample)
    df_val = df_val.sample(frac = sample)
    
    df_trn = merge_cols(df_trn)
    df_val = merge_cols(df_val)
    
    db_trn = conv_to_ds(df_trn, data_path)
    db_val = conv_to_ds(df_val, data_path)
        
    data = TextLMDataBunch.create(
        train_ds = db_trn.train_ds, valid_ds = db_val.train_ds, path = data_path, bs = bs
    )
    
    data.label_list = db_trn.label_list
    data.label_list.valid = db_val.label_list.train

    return data

In [ ]:
df_trn, df_val, df_tst = read_data(data_path/"merged")

In [ ]:
df_trn["query"][10], df_trn["res"][10]

In [51]:
data = gen_lm_data(df_trn, df_val, "merged", data_path, sample = 0.01)

In [52]:
data.label_list

LabelLists;

Train: LabelList (4929 items)
x: LMTextList
▁x x bo s ▁i ▁use ▁image j ▁1 . x ▁x x up ▁api ▁in ▁my ▁x x ma j ▁java ▁application , ▁where ▁i ▁run ▁image j ▁in ▁in visible ▁mode : ▁image j ▁image j application ▁= ▁new ▁image j ( 2); ▁x x ma j ▁in ▁my ▁x x ma j ▁java ▁application ▁i ▁generate ▁several ▁images ▁and ▁open ▁them ▁for ▁the ▁user ▁inter action s : ▁x x ma j ▁open er ▁open er ▁= ▁new ▁x x ma j ▁open er (); ▁x x ma j ▁string ▁image file path ▁= ▁get last image file path (); ▁image plus ▁i mp ▁= ▁open er . open image ( image file path ); ▁i mp . show (); ▁i ▁provide ▁a ▁user ▁with ▁the ▁pos s ibility ▁to ▁make ▁a ▁str a ight ▁line ▁selection ▁on ▁any ▁of ▁the ▁open ed ▁images : ▁i j . set to ol ( 4 ); ▁x x ma j ▁if ▁the ▁user ▁has ▁draw n ▁a ▁str a ight ▁line ▁on ▁an ▁image , ▁then ▁i ▁need ▁to ▁get ▁length ▁of ▁this ▁line . ▁x x ma j ▁to ▁do ▁this ▁i ▁try ▁to ▁get ▁access ▁to ▁the ▁str a ight ▁line ▁selection ▁object ▁that ▁was ▁produce d ▁by ▁the ▁user ▁inter actio

In [ ]:
len(data.train_ds), len(data.valid_ds)

In [ ]:
data.show_batch()

In [44]:
data.save("test_data.pkl")

In [45]:
tst_data = load_data(data_path, "test_data.pkl", bs = 32)

In [46]:
tst_data

TextLMDataBunch;

Train: LabelList (4929 items)
x: LMTextList
▁x x bo s ▁ jms j ca managed connection ▁get managed connection () ▁throws ▁ illegal state exception ▁{ ▁if ▁( trace component . is any tra c ing enabled () ▁&& ▁trace . is entry enabled ()) ▁{ ▁ sib tr . entry ( this , ▁x x up ▁trace , ▁" get managed connection "); ▁} ▁if ▁( _ managed connection ▁== ▁null ) ▁{ ▁final ▁connection manager ▁connection manager ▁= ▁ _ connection ▁ . get connection manager (); ▁if ▁( connection manager ▁== ▁null ) ▁{ ▁throw ▁new ▁ illegal state exception ( nl s . get format ted message ( ▁x x up ▁ (" exception _ receive d _ c ws j r 11 26 "), ▁new ▁x x ma j ▁object [] ▁{ ▁" get managed connection "}, ▁null )); ▁} ▁if ▁( connection manager ▁instanceof ▁lazy as s oc ia table connection manager ) ▁{ ▁try ▁{ ▁final ▁managed connection factory ▁managed connection factory ▁= ▁ _ connection ▁ . get connection factory () ▁ . get managed connection factory (); ▁( ( la z y as s oc ia table connection manag

# Bunch of Debugging

In [ ]:
db1 = db.dl(DatasetType.Train)

In [ ]:
 db2 = TextLMDataBunch.create(
        train_ds = db.train_dl.dl.dataset,
        valid_ds = db.train_dl.dl.dataset, path = data_path
    )

In [ ]:
db2.show_batch()

In [ ]:
db.show_batch()

In [ ]:
data.show_batch()

In [ ]:
db1 = db.databunch(bs=32)

In [ ]:
db1.show_batch()

In [ ]:
d1 = (TextList
            .from_df(df_trn[:100], data_path, 
                     processor = SPProcessor(
                         sp_model = data_path/"merged/model.model",
                         sp_vocab = data_path/"merged/model.vocab"
                     ))
            .split_none()
            .label_for_lm()
#             .databunch(bs=64)
           )

In [ ]:
d2 = d1.databunch(bs=64)
d2.show_batch()

In [ ]:
db.show_batch()

In [ ]:
data.train_dl.dl.dataset[5]

In [ ]:
new_data = (TextList
            .from_df(df_trn[:100], data_path, 
                     processor = SPProcessor(
                         sp_model = data_path/"merged/model.model",
                         sp_vocab = data_path/"merged/model.vocab"
                     ))
            .split_none()
            .label_for_lm()
            .databunch(bs=64)
           )

In [ ]:
new_data.show_batch()

In [ ]:
new_data.train_dl.dl.dataset[5]